# Explanation

U-Net takes advantage of the CNN architecture and data augmentation to create a highly effective image segmentation model with very little data. This is interesting because deep learning usually takes a lot of data - but U-Net is an example of solving a challenging problem with inductive bias, despite low data.

I've included U-Net here not just for it's independent value but also its practical value - it's now the model of choice in many modern image generation architectures we'll cover in section 5.

# My Notes

📜 [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597)


> There is large consent that successful training of deep networks  requires many thousand annotated training samples. In this paper, we present a network and training strategy that relies on the strong use of data augmentation to use the available annotated samples more efficiently.
> 

This paper shows us that it’s possible to train effective deep neural networks even with a small amount of data, given their new approaches.

> The architecture consists of a contracting path to capture
context and a symmetric expanding path that enables precise localization.
> 

> We show that such a network can be trained end-to-end from very
few images and outperforms the prior best method.
> 

> The main idea in to supplement a usual contracting network by
successive layers, where pooling operators are replaced by upsampling operators.
> 

> In order to localize, high resolution features from the contracting path are combined with the upsampled output.
> 

> One important modification in our architecture is that in the upsampling part we have also a large number of feature channels, which allow the network to propagate context information to higher resolution layers.
> 

> As for our tasks there is very little training data available, we use excessive
data augmentation by applying elastic deformations to the available training images.
> 

### Network Architecture

![Screenshot 2024-05-22 at 2.48.43 AM.png](../../images/Screenshot_2024-05-22_at_2.48.43_AM.png)

### Training

**3.1 Data Augmentation**

> We generate smooth deformations using random displacement vectors on a coarse 3 by 3 grid.
> 

They use data augmentation to successfully modify the dataset and make it much larger, letting them improve the model with very few available training samples.

### Experiments

![Screenshot 2024-05-22 at 3.02.05 AM.png](../../images/Screenshot_2024-05-22_at_3.02.05_AM.png)

### Conclusion

> Thanks to data augmentation with elastic deformations, it only needs very few annotated images and has a very reasonable training time of only 10 hours on a NVIDIA Titan GPU
>